In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import *

# Build Data - A Assignments

In [2]:
QUOTE_DATE = '2022-03-31'
filepath_rawdata = f'../data/treasury_quotes_{QUOTE_DATE}.xlsx'
rawdata = pd.read_excel(filepath_rawdata,sheet_name='quotes')
rawdata.columns = rawdata.columns.str.upper()
rawdata.sort_values('TMATDT',inplace=True)
rawdata.set_index('KYTREASNO',inplace=True)

t_check = rawdata['CALDT'].values[0]
if rawdata['CALDT'].eq(t_check).all():
    t_current = t_check
else:
    warnings.warn('Quotes are from multiple dates.')
    t_current = None

display(f'Treasuries quoted on {QUOTE_DATE}.')

'Treasuries quoted on 2022-03-31.'

In [3]:
mats = get_maturity_delta(rawdata['TMATDT'],t_current=t_current)
targs = pd.Series(np.arange(.5,7.5,.5))

idbonds = list()
for tau in targs:
    idbonds.append((mats-tau).abs().idxmin())

metrics = rawdata.copy()[['TDATDT','TMATDT','TCOUPRT','TDYLD','TDDURATN']]
metrics.columns = ['issue date','maturity date','coupon rate','ytm','duration']
metrics['ytm'] *= 365
metrics['duration'] /= 365
metrics['maturity'] = get_maturity_delta(metrics['maturity date'], t_current)
metrics['price'] = (rawdata['TDBID'] + rawdata['TDASK'])/2 + rawdata['TDACCINT']
metrics = metrics[['issue date','maturity date','maturity','coupon rate','price','ytm']]

mets = metrics.loc[idbonds,:]
mets

,issue date,maturity date,maturity,coupon rate,price,ytm
KYTREASNO,,,,,,
206896,2015-09-30,2022-09-30,0.501027,1.750,100.363281,0.010144
207688,2021-03-31,2023-03-31,0.999316,0.125,98.457031,0.016804
207775,2021-09-30,2023-09-30,1.500342,0.250,97.218750,0.021307
207077,2017-03-31,2024-03-31,2.001369,2.125,99.570312,0.023294
207416,2019-09-30,2024-09-30,2.502396,1.500,97.621094,0.024680
207507,2020-03-31,2025-03-31,3.000684,0.500,94.175781,0.025101
207256,2018-09-30,2025-09-30,3.501711,3.000,101.554688,0.025142
207337,2019-03-31,2026-03-31,4.000000,2.250,98.929688,0.025154
207777,2021-09-30,2026-09-30,4.501027,0.875,93.042969,0.025020


In [5]:
outfile = '../data/selected_treasury_rates.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    mets.to_excel(writer, sheet_name= 'data', index=True)